### Installation

In [1]:
%pip install torch
%pip install numpy
%pip install transformers
%pip install pandas
%pip install ipywidgets
%pip install tqdm

     |████████████████████████████████| 119.6 MB 6.2 kB/s  eta 0:00:01
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
     |████████████████████████████████| 16.1 MB 29.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     |████████████████████████████████| 2.1 MB 9.0 MB/s eta 0:00:01
     |████████████████████████████████| 75 kB 11.3 MB/s eta 0:00:01
     |████████████████████████████████| 2.3 MB 47.8 MB/s eta 0:00:01
     |████████████████████████████████| 862 kB 32.0 MB/s eta 0:00:01
     |████████████████████████████████| 284 kB 25.4 MB/s eta 0:00:01
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
     |████████████████████████████████| 303 kB 36.9 MB/s eta 0:00:01
Using legacy 'setup.py install' for sacremoses, since package 'wheel' is not installed.
    Running setup.py install for sacremoses ... done
Note: you may need to restart the kernel to use upda

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Imports

In [2]:
import requests
import transformers
from transformers import RobertaTokenizer, RobertaModel
from ipywidgets import IntProgress
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm import tqdm

### Download data

We'll be using the _Offensive Language Identification Dataset (OLID)_ dataset to specialize BERT in hate speech detection. You can find the dataset [here](https://scholar.harvard.edu/malmasi/olid). The following code will download it into the current dir.

In [3]:
!git clone https://github.com/idontflow/OLID.git

Cloning into 'OLID'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 35 (delta 18), reused 28 (delta 11), pack-reused 0
Unpacking objects: 100% (35/35), done.


### Examine our data

OLID (2019) is comprised of a number of datasets. Let's break them down now. olid-training-v1.tsv has ~13,000 annotated tweets and 3 subtask labels. Then we have our test sets for each subtask: testset-levela.tsv, testset-levelb.tsv, testset-levelc.tsv.

subtask_a is a categorization on whether the tweet is offensive. Here are the categories:

- (NOT) Not Offensive - This post does not contain offense or profanity.
- (OFF) Offensive - This post contains offensive language or a targeted (veiled or direct) offense

If a tweet was labeled as offensive, then it can have a value for subtask B. Here are the categories for subtask_b:

- (TIN) Targeted Insult and Threats - A post containing an insult or threat to an individual, a group, or others (see categories in sub-task C).
- (UNT) Untargeted - A post containing non-targeted profanity and swearing.

If a tweet was marked as offensive and targeted, then it can have the following categories for subtask_c:

- (IND) Individual - The target of the offensive post is an individual: a famous person, a named individual or an unnamed person interacting in the conversation.
- (GRP) Group - The target of the offensive post is a group of people considered as a unity due to the same ethnicity, gender or sexual orientation, political affiliation, religious belief, or something else.
- (OTH) Other – The target of the offensive post does not belong to any of the previous two categories (e.g., an organization, a situation, an event, or an issue)

Of course, if a tweet isn't offensive, it won't have values for subtask b and c. Similarly, if the tweet is untargeted, it won't have a value for subtask c. 

Now, let's take a look at the dataset. 

In [4]:
data = pd.read_csv('./OLID/olid-training-v1.0.tsv', sep='\t')
data.head()

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN


We want to make sure we can run this code on CUDA, if available.

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

Let's start by making a classifier for generally offensive tweets (subtask_a). We'll make a pytorch model that extends roBERTa.

### subtask_a

We'll be using roBERTa's tokenizer to encode our text. Behind the scene, it's doing byte-level byte pair encoding (BPE). Let's also define some constants here.

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base", truncation=True)
batch_size = 32
train_size = 0.8 
max_len = 280
epochs = 10

Let's clean up our dataframe by converting "OFF" to 1 and "NOT" to 0. 

In [19]:
data.loc[data["subtask_a"] == "OFF", "subtask_a"] = 1
data.loc[data["subtask_a"] == "NOT", "subtask_a"] = 0
data.head()

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,1,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,1,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,0,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",1,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,0,NaN,NaN


Let's define a Data class for this task. We'll extend pytorch's Dataset class here so we can initialize DataLoaders with it down the road. 

In [24]:
class DataA(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        # Our data
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        # X and Y
        self.text = self.data.tweet
        self.targets = self.data.subtask_a
    
    # Some class methods we have to override:
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, index):
        # Extract the text at index
        text = str(self.text[index])
        text = "".join(text.split())
        
        # Create a dictionary that contains the encoded sequence
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            return_token_type_ids=True,
        )
        
        # Extract the values we care about from inputs
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }   

Let's make our train/test sets!!

In [25]:
# Sample from our dataset 
train_data = data.sample(frac=train_size, random_state=0)
# Remove the train data we sampled in the last step to get our training data
test_data = data.drop(train_data.index).reset_index(drop=True)
# Clean up train data by resetting the indexes
train_data = train_data.reset_index(drop=True)

# Our train and test data shape
print("Train shape:", train_data.shape)
print("Test shape:", test_data.shape)

# Our data
train_set = DataA(train_data, tokenizer, max_len)
test_set = DataA(test_data, tokenizer, max_len)

# Our dataloaders
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=0)

Train shape: (10592, 5)
Test shape: (2648, 5)


Our model:

In [26]:
class ModelA(torch.nn.Module):
    def __init__(self):
        super(ModelA, self).__init__()

        # Layer 1 is Roberta
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        # First hidden layer for our classifier
        self.l2 = torch.nn.Linear(768, 768)
        # ReLU
        self.relu = torch.nn.ReLU() 
        # Dropout
        self.dropout = torch.nn.Dropout(0.5)
        # Output layer
        self.l3 = torch.nn.Linear(768, 2)
        # Softmax
        self.softmax = torch.nn.Softmax(dim=1)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        # Roberta inputs
        x = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        x = self.l2(x[0][:, 0])
        x = self.relu(x)
        x = self.dropout(x)
        x = self.l3(x)
        x = self.softmax(x)
        return x

In [33]:
def calculate_accuracy(pred, target):
    n_correct = (pred==targets).sum().item()
    return n_correct

Let's train this jawn

In [36]:
modelA = ModelA()
modelA.to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer_fn = torch.optim.Adam(params =  modelA.parameters(), lr=1e-3)

In [ ]:
for epoch in range(epochs):
    curr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    modelA.train()
    # Loop over batch data
    # btw tqdm is a nice loop progress visualizer
    for step, example in tqdm(enumerate(train_loader, 0)):
        # Convert example from batch into input for Roberta
        ids = example['ids'].to(device, dtype = torch.long)
        mask = example['mask'].to(device, dtype = torch.long)
        token_type_ids = example['token_type_ids'].to(device, dtype = torch.long)
        targets = example['targets'].to(device, dtype = torch.long)
        
        # Get model outputs
        outputs = modelA(ids, mask, token_type_ids)
        loss = loss_fn(outputs, targets)
        # Add loss to current epoch loss tracker
        curr_loss += loss.item()
        
        # Add to our counters
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calculate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        # Calculate backward pass
        optimizer_fn.zero_grad()
        loss.backward()
        # Update parameters
        optimizer_fn.step()
        if step % 5000 == 0:
            loss_step = curr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")
        
    print(f'The total accuracy for epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = curr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {epoch_accu}")
    
        

1it [01:28, 88.08s/it]

Training Loss per 5000 steps: 0.6616766452789307
Training Accuracy per 5000 steps: 84.375


331it [7:46:47, 84.61s/it]
0it [00:00, ?it/s]

The total accuracy for epoch 0: 66.32364048338368
Training loss epoch: 0.6477830762949595
Training accuracy epoch: 66.32364048338368


1it [01:23, 83.26s/it]

Training Loss per 5000 steps: 0.677889347076416
Training Accuracy per 5000 steps: 62.5


121it [3:33:01, 109.57s/it]

Now, let's save our model

In [ ]:
torch.save(modelA, 'hate_speech_model.bin')